In [1]:
# Standard python packages
import os
import numpy as np
import pandas as pd

# LavAtmos 2: Example 1 
### Running LavAtmos 2 for a Bulk Silicate Earth composition for a single surface temperature and total volatile pressure

In this notebook we will show how straightforward it is to use LavAtmos 2 in order to calculate the partial pressures of the vapor species above a melt of bulk silicate earth (BSE) composition under a volatile atmosphere. Start by ensuring that the standard python packages imported above are all present in your build. Then make sure that the path to the LavAtmos module in the lines below is correct. If you are running the code in the docker provided for Thermoengine on the [ENKI](https://enki-portal.gitlab.io/ThermoEngine/) page, then you should not have to change the paths. 

In [2]:
os.chdir('/home/jovyan/ThermoEngine/LavAtmos')
import lavatmos2

### Setting the melt composition
Next up, we import the bulk silicate earth composition that was also used in the [Visscher and Fegley (2013) paper](https://doi.org/10.1088/2041-8205/767/1/L12). You may also chose to enter your own composition by either change the data file from which we are importing, or by defining a new dictionary type object with the weight percentages for each included oxide. If you do the latter, ensure that the dictionary is in the same format as the one printed below. 

In [3]:
vf13_comps_df = pd.read_csv('/home/jovyan/ThermoEngine/LavAtmos/data/input/vf2013_comps.csv',index_col=0)
vf13_comps = {}
for name in vf13_comps_df.columns:
    vf13_comps[name] = vf13_comps_df[name].to_dict()
print(vf13_comps['BSE'])

{'SiO2': 45.4, 'MgO': 36.76, 'Al2O3': 4.48, 'TiO2': 0.21, 'FeO': 8.1, 'CaO': 3.65, 'Na2O': 0.349, 'K2O': 0.031}


### Setting the volatile atmosphere composition
We now determine the volatile atmosphere composition. In this example we use a hydrogen (H) composition that is roughly similar to what you would expect for an atmosphere with a solar-like composition. Similarly to the melt composition, the volatile compostion is given in dictionary format. Note that here we pass it on in terms of mole fractions (maximum of 1). Feel free to try different compositions yourself!

In [4]:
volatile_comp = {
    'C' : 0.01,
    'H' : 0.80,
    'N' : 0.18,
    'S' : 0.005,
    'P' : 0.005
    }

### Setting the surface temperature and the total volatile pressure of the system

Lets try a surface temperature of 3000 K and a total volatile atmosphere pressure of 10 bar.

Note: The total volatile pressure is the total pressure of all the volatile species in the atmosphere, but not of the atmosphere as a whole. The total pressure as a whole is determined by adding the total volatile pressure and the pressure of all the vaporised species to each other. 

In [5]:
T_surf = 2300 # Kelvin - if you want to do multiple values, make sure that it is an np.array()
P_vol = 10 # Bar

### Initialising LavAtmos

Next up we initialise a LavAtmos system. This makes sure that the necessary thermochemical data is imported and loaded. 

In [6]:
system = lavatmos2.melt_vapor_system()

### Running LavAtmos

Now we have all we have to do is run the `vaporise` from system by passing on the temperature `T` and the compositions.

In [7]:
partial_pressures = system.vaporise(T_surf, P_vol, vf13_comps['BSE'], volatile_comp)

Calculated:   0%|          | 0/1 [00:00<?, ?it/s]
Point falls within 200 K and 1 order of magnitude of cached values.
Using extrapolation for first estimate of fO2.
Estimated value: 0.0001684567986162122
Calculated: 100%|██████████| 1/1 [04:40<00:00, 280.02s/it]


In [8]:
print(partial_pressures)

              C         H            He             N             O         P  \
0  4.110189e-12  0.026739  2.029133e-31  5.175628e-08  1.708996e-07  0.000542   

          S        Si            Ti             V  ...           P1+  \
0  0.000377  0.000002  3.416187e-10  5.682120e-33  ...  2.511083e-19   

            P1-           S1+           S1-          Si1+          Si1-  \
0  2.124710e-09  8.808417e-20  3.853880e-07  3.332472e-17  3.600452e-11   

           Ti1+          Ti1-           V1+           V1-  
0  1.911836e-17  2.841556e-17  2.093811e-40  2.472305e-39  

[1 rows x 473 columns]
